In [1]:
import pandas as pd
import numpy as np

import time
from os import getcwd 
from os.path import exists

getcwd() # current working directory

'D:\\project\\MIT_glyco'

In [2]:
version = 'v2'
update = False

In [3]:
### This part is grouping the positive sites of protein with same name ###
time_start = time.time()
# load O-GlcNAcylated site data
site_data = pd.read_csv("./OGP-master/all_sites_with_colnames.csv") 

columns = ["name", "site", "sequence"]
names = site_data["name"]
sites = site_data["site"]
sequences = site_data["sequence"]
N = len(site_data)

sites_data = pd.DataFrame([], columns=columns)
save_name = "all_sites_group.csv"
if not exists(save_name) or update:
    # grouping positive sites of proteins with same name
    for i in range(N):
        if i==0:
            pos_sites = [sites[i]]
            temp_df = pd.DataFrame([[names[i], pos_sites, sequences[i]]], columns=columns)
        elif names[i] != names[i-1]: 
            sites_data = pd.concat([sites_data, temp_df], axis=0)
            pos_sites = [sites[i]]
            temp_df = pd.DataFrame([[names[i], pos_sites, sequences[i]]], columns=columns)
        else:
            pos_sites.append(sites[i])
            if i == N-1:
                sites_data = pd.concat([sites_data, temp_df], axis=0)
            
    # reset index
    sites_data = sites_data.reset_index(drop=True)
    
    # save file
    sites_data.to_csv(save_name, index=False)

else: # if the file already exists, then load the file 
    sites_data = pd.read_csv(save_name)
    
time_end = time.time()
executed_time = (time_end - time_start)/60
print(f'executed time: {executed_time:.2f} min')

display(sites_data)
## executed time = 0.01 min

executed time: 0.00 min


,name,site,sequence
0,A2ABU4,[430],MTLPHSPGSAGEPQASQTVQVHRLEHRQEEEQKEERQHSLQMGSSV...
1,A2AGT5,[568],MGDDSEWLKLPVDQKCEHKLWKARLSGYEEALKIFQKIKDEKSPEW...
2,A2AHJ4,"[264, 279]",MAAAPTQIEAELYYLIARFLQSGPCNKSAQVLVQELEEHQLIPRRL...
3,A2AKB9,"[87, 99, 100, 364]",MFPFGPHSPGGDETAGAEEPPPLGGPAAASRPPSPAPRPASPQRGA...
4,E9Q1P8,[208],MAAAVAVAAASRRQSCYLCDLPRMPWAMIWDFTEPVCRGCVNYEGA...
...,...,...,...
281,Q9P2N5,[738],MLIEDVDALKSWLAKLLEPICDADPSALANYVVALVKKDKPEKELK...
282,Q9UPN6,"[615, 614]",MEAVKTFNSELYSLNDYKPPISKAKMTQITKAAIKAIKFYKHVVQS...
283,Q9UQ35,[2236],MYNGIGLPTPRGSGTNGYVQRNLSLVRGRRGERPDYKGEEELRRLE...
284,Q9Y2X9,[891],MKIGSGFLSGGGGTGSSGGSGSGGGGSGGGGGGGSSGRRAEMEPTF...


In [4]:
### This part generates dataframe including protein sequence, flexibility, fichier information of each protein ###
time_start = time.time()

flex_path = "./OGP-master/dynamine_results" # directory of flexibility results
angle_path = "./OGP-master/spider3_results" # directory of angle related information
N = len(sites_data)

name = sites_data['name']
site = sites_data['site']

pass_list = ["P24622_2", "Q91YE8_2"] #these proteins have positive sites which are out of bound

save_path = './protein_sequence'
for i in range(N): # protein name & positive site
    pro_name = name[i]
    if pro_name in pass_list: # if the protein is in pass_list, then pass the below process
        continue
        
    save_name = f"{save_path}/{pro_name}.csv"
    if not exists(save_name) or update:
        site_index = [j-1 for j in site[i]]
        ## load flex data
        flex_data = pd.read_fwf(f"{flex_path}/{pro_name}_backbone.pred", header=None)
        notes = flex_data[:11] # this includes protein name and citation address
        # display(notes)
        ## split values from raw flexibility data
        flex = flex_data[11:].copy().reset_index(drop=True)
        flex.columns = ["values"]
        flex['SEQ'] = flex['values'].apply(lambda x: x.split()[0])
        flex['flexibility'] = flex['values'].apply(lambda x: x.split()[1])

        ## load angle data
        angles = pd.read_csv(f"{angle_path}/{pro_name}.spd33")
        ## split values from raw angle data
        columns = angles.columns[0].split()
        angles.columns = ["values"]

        for k, column in enumerate(columns):
            angles[column] = angles['values'].apply(lambda x: x.split()[k])

        flex_values = flex.iloc[:,1:]
        angle_values = angles.iloc[:,3:]
        protein_data = pd.concat([flex_values, angle_values], axis=1)

        protein_data.index.name = f"{pro_name}, {site[i]}"
        
        # assign positive value if the site is glycosylated
        protein_data["positive"] = 0
        protein_data.loc[site_index, "positive"] = 1
        
        # allign columns
        columns = protein_data.columns
        new_columns = [columns[0]] + [columns[-1]] + list(columns[1:-1])
        protein_data = protein_data[new_columns]

        # save the dataframe
        protein_data.to_csv(save_name)
        
    else:
        protein_data = pd.read_csv(save_name, index_col=0)
        
time_end = time.time()
executed_time = (time_end - time_start)/60
print(f'executed time: {executed_time:.2f} min')

display(protein_data)
## executed time = 0.13 min

executed time: 0.13 min


,SEQ,positive,flexibility,SS,ASA,Phi,Psi,Theta(i-1=>i+1),Tau(i-2=>i+2),HSE_alpha_up,HSE_alpha_down,P(C),P(H),P(E)
"Q9Y520, [2693, 2243]",,,,,,,,,,,,,,
0,M,0,0.422,C,155.0,-95.8,133.1,115.4,165.4,3.1,3.3,0.988,0.007,0.005
1,S,0,0.402,C,103.7,-97.1,140.5,116.6,-157.5,1.4,3.8,0.952,0.018,0.029
2,E,0,0.414,C,151.2,-89.7,125.5,111.2,-135.5,2.9,7.7,0.918,0.031,0.051
3,K,0,0.434,C,151.6,-89.7,115.7,109.8,-160.5,3.5,6.9,0.915,0.038,0.047
4,S,0,0.453,C,92.5,-87.4,115.3,109.1,-128.5,2.4,7.0,0.939,0.034,0.028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2891,E,0,0.580,C,110.1,-95.9,129.5,114.1,-151.5,8.1,12.4,0.686,0.253,0.062
2892,E,0,0.540,C,129.9,-89.6,124.2,111.0,-140.1,5.4,11.0,0.713,0.227,0.061
2893,T,0,0.473,C,106.7,-91.4,100.3,109.1,-163.0,3.1,9.1,0.753,0.168,0.080


In [5]:
### This part generates sequential data within the fixed-length window ###
# X1: Glycosylated or not
# X2: the number of serine and threonine in the -10 to +10 window
# X3: the number of aliphatic residues in the window -3 to -1 window
# X4: the number of positively charged residue in the -7 to -5 window
# X5: side chain length -1 to 5 window (1 no residue, 2 glysine, 3 very small, 4 small, 5 long, 6 cycle, 7 proline)
# X6: flexibility of the site
# X7: type of site nature (serine or threonine)
# X8: second structure (1 not structured, 2 alpha helix, 3 beta strand)
# X9: secondary structure according to phi and psi angles (1 beta, 2 alpha, 3 other)

def make_window(index, protein_data, window, marking=False):
    window_start = max(index+window[0], 0)
    window_end   = min(index+window[1]+1, len(protein_data))
    if marking:
        sequence = protein_data['SEQ'].iloc[window_start:window_end].copy()
        sequence.iloc[index-window_start] = f'"{sequence.iloc[index-window_start]}"'
        sequence = sequence.sum()        
    else:
        sequence = protein_data['SEQ'].iloc[window_start:window_end].sum()
    return sequence

def check_length(letter):
    if (letter == "G"): #1
        return 'gly' 
    elif (letter == "V" or letter == "A"): #2 Asn, Gln
        return 'very small'
    elif (letter == "S" or letter == "I" or letter == "L" or letter == "T" or letter == "C"): #3 Ser, Thr, Ile, Leu, Cys
        return 'small'
    elif (letter == "D" or letter == "E" or letter == "N" or letter == "Q" or letter == "M"): #4 Asp, Asn, Glu, Gln, Met
        return 'normal'
    elif (letter == "R" or letter == "K"): #5 Arg, Lys
        return 'long'
    elif (letter == "F" or letter == "W" or letter == "Y" or letter == "H"): #6 Phe, Trp, Tyr, His
        return 'cycle'
    elif (letter == "P"): #7
        return 'pro'
    else:
        return 'other' #0

class Protein():
    def __init__(self, protein_data):
        self.protein_data = protein_data.copy()
        self.N = len(protein_data)
        
    def count_ST(self, index, window):
        sequence = make_window(index, self.protein_data, window)
        nS = sequence.count("S")
        nT = sequence.count("T")
        if self.protein_data['SEQ'].iloc[index] == 'S':
            nS -= 1
        if self.protein_data['SEQ'].iloc[index] == 'T':
            nT -= 1
        return nS, nT

    def count_aliphaticAA(self, index, window): # count the number of aliphatic AA(Ala, Val, Leu, Ile, Pro)
        sequence = make_window(index, self.protein_data, window)
        nA = sequence.count("A")
        nV = sequence.count("V")
        nL = sequence.count("L")
        nI = sequence.count("I")
        nP = sequence.count("P")
        return nA + nV + nL + nI + nP

    def count_positiveAA(self, index, window): # count the number of positively charged AA (Ard, Lys, His)
        sequence = make_window(index, self.protein_data, window)
        nR = sequence.count("R")
        nK = sequence.count("K")
        nH = sequence.count("H")
        return nR + nK + nH
    
    def is_pro_after(self, index, after): # check whether there is a proline after the site
        return self.protein_data['SEQ'].iloc[index+after] == 'P'
    
    def check_angle(self, index):
        phi, psi = self.protein_data[['Phi','Psi']].iloc[index]
        if phi > -160 and phi < -50:
            if psi > 100 and psi < 180:
                return "alpha"
            elif psi > -60 and psi < 20:
                return "beta"
        return "other"


In [6]:
time_start = time.time()
columns = ['window', 'nSer', 'nThr', 'nAli', 'nPos', 'P_after', 'SS_angle']
protein_columns = ['ASA', 'Phi', 'Psi', 'Theta(i-1=>i+1)', 'Tau(i-2=>i+2)', 'HSE_alpha_up', 'HSE_alpha_down', 
                   'P(C)', 'P(H)', 'P(E)']
# Side chain length
side_window = [-1, 5]
side_range = range(side_window[0], side_window[1]+1)
side_columns = [f"side{i}" for i in side_range if i != 0]

new_columns = ["name", 'index', 'SEQ', 'positive', 'window', 
               'nSer', 'nThr', 'nAli', 'nPos', 'P_after', 
               'flexibility', 'SS', 'SS_angle'] + side_columns + protein_columns

save_name = f'data_for_ml_{version}.csv'
if not exists(save_name) or update:
    for i in range(N): # protein name & positive site

        pro_name = name[i]
        if pro_name in pass_list: # if the protein is in pass_list, then pass the below process
            continue
        load_name = f"./protein_sequence/{pro_name}.csv"
        protein_data = pd.read_csv(load_name, index_col=0)

        # protein object
        protein = Protein(protein_data)
        protein_data['name']  = pro_name
        protein_data[columns] = 0
        protein_data[side_columns] = 0


        # find index of serine or threonine in the sequence
        ST_index = np.where( (protein_data['SEQ']=='S') | (protein_data['SEQ']=='T') )[0]
        for index in ST_index:
            # count the number of serine and threonine
            window = [-10,10]
            protein_data.loc[index, "window"] = make_window(index, protein_data, window, marking=True)

            nS, nT = protein.count_ST(index, window)
            protein_data.loc[index, "nSer"] = nS
            protein_data.loc[index, "nThr"] = nT

            # count the number of aliphatic AA(Ala, Val, Leu, Ile, Pro)
            window = [-3, -1]
            check_min = max(index+window[0], 0)
            check_max = min(index+window[1]+1, len(protein_data))
            if check_max > 0: # check whether window exists
                protein_data.loc[index, "nAli"] = protein.count_aliphaticAA(index, window)

            # count the number of positively charged AA (Ard, Lys, His)
            window = [-7, -5]
            check_min = max(index+window[0], 0)
            check_max = min(index+window[1]+1, len(protein_data))
            if check_max > 0: # check whether window exists
                protein_data.loc[index, "nPos"] = protein.count_positiveAA(index, window)

            # check whether there is a proline after the site
            after = 1
            check_max = index+after
            if check_max < len(protein_data):
                protein_data.loc[index, "P_after"] = protein.is_pro_after(index, after)

            # Side chain length
            sides = range(index+side_window[0], index+side_window[1]+1)
            for j, side in enumerate(sides):
                if side == index:
                    pass
                elif side < 0 or side >= len(protein_data):
                    pass
                else:
                    protein_data.loc[index, f"side{side_range[j]}"] = check_length(protein_data['SEQ'].iloc[side])

            # SS by angle
            protein_data.loc[index, "SS_angle"] = protein.check_angle(index)

        if i == 0:
            data_for_ml = protein_data.iloc[ST_index]
        else:
            data_for_ml = pd.concat([data_for_ml, protein_data.iloc[ST_index]], axis=0)

    data_for_ml = data_for_ml.reset_index(drop=False)
    data_for_ml = data_for_ml[new_columns]
    data_for_ml.to_csv(save_name)

else:
    data_for_ml = pd.read_csv(save_name, index_col=0)

time_end = time.time()
executed_time = (time_end - time_start)/60
print(f'executed time: {executed_time:.2f} min')

display(data_for_ml)
## executed time = 1.00 min

executed time: 1.00 min


,name,index,SEQ,positive,window,nSer,nThr,nAli,nPos,P_after,...,ASA,Phi,Psi,Theta(i-1=>i+1),Tau(i-2=>i+2),HSE_alpha_up,HSE_alpha_down,P(C),P(H),P(E)
0,A2ABU4,1,T,0,"M""T""LPHSPGSAGE",2,0,0,0,False,...,103.3,-102.0,132.1,117.6,-150.0,3.8,13.9,0.839,0.011,0.150
1,A2ABU4,5,S,0,"MTLPH""S""PGSAGEPQAS",2,1,2,0,True,...,60.0,-87.4,138.5,115.2,-125.9,7.8,16.7,0.925,0.005,0.070
2,A2ABU4,8,S,0,"MTLPHSPG""S""AGEPQASQTV",2,2,1,0,False,...,56.1,-89.9,142.4,116.8,121.2,8.2,13.9,0.895,0.039,0.067
3,A2ABU4,15,S,0,"SPGSAGEPQA""S""QTVQVHRLEH",2,1,2,0,False,...,75.5,-82.7,22.5,104.9,-107.4,5.9,14.2,0.681,0.295,0.023
4,A2ABU4,17,T,0,"GSAGEPQASQ""T""VQVHRLEHRQ",2,0,1,0,False,...,78.2,-96.3,112.1,112.0,84.6,5.8,13.7,0.445,0.233,0.322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42815,Q9Y520,2875,T,0,"EKPPPAPSIA""T""KPVRTGPIKP",1,1,2,0,False,...,76.2,-95.6,138.6,117.8,-135.9,8.8,13.9,0.794,0.097,0.110
42816,Q9Y520,2880,T,0,"APSIATKPVR""T""GPIKPQAIKT",1,2,2,0,False,...,58.2,-99.5,90.7,111.0,-161.6,11.6,16.2,0.865,0.014,0.120
42817,Q9Y520,2890,T,0,"TGPIKPQAIK""T""EETKS",1,2,2,1,False,...,80.3,-102.2,131.1,116.5,-164.4,7.0,14.3,0.662,0.216,0.122
42818,Q9Y520,2893,T,0,"IKPQAIKTEE""T""KS",1,1,0,0,False,...,106.7,-91.4,100.3,109.1,-163.0,3.1,9.1,0.753,0.168,0.080
